# Pizza pizza pizza

Get top venues by keyword and location, querying Google, Yelp, Foursquare

#### Google

 - Needs a Google API key and module
 - [Create Google Cloud credentials and give access to Places APIs](https://console.cloud.google.com/google/maps-apis/credentials)
 - `conda install -c conda-forge -y gmaps`
 - put key in `apikey.txt`
 - `gmaps` Jupyter nbextension to show maps in notebook, with marker pins etc.

```
conda install -c conda-forge -y jupyter_contrib_nbextensions
jupyter nbextension enable --py gmaps
jupyter notebook
```

#### Yelp
 - needs Yelp API key and module
 - https://www.yelp.com/developers/documentation/v3
 - https://github.com/gfairchild/yelpapi
 - put key in `yelpkey.txt`
 
#### Foursquare
- Needs Foursquare API key and module
- https://developer.foursquare.com/docs/places-api/getting-started/
- https://github.com/mLewisLogic/foursquare
- OAuth id in `foursquare_id.txt`
- OAuth secret in `foursquare_secret.txt`

See `requirements.txt` for versions used, other requirements (requests, folium, Flask)


In [1]:
import time
from pprint import pprint
import pdb

import pandas as pd

import requests, json 

import gmaps
with open('apikey.txt') as f:
    api_key = f.readline().strip()
    f.close
gmaps.configure(api_key=api_key)

# https://github.com/gfairchild/yelpapi
from yelpapi import YelpAPI
with open('yelpkey.txt') as f:
    yelp_key = f.readline().strip()
    f.close
yelp_api = YelpAPI(yelp_key)

import foursquare
with open('foursquare_id.txt') as f:
    foursquare_id = f.readline().strip()
    f.close
with open('foursquare_secret.txt') as f:
    foursquare_secret = f.readline().strip()
    f.close

gmaps.configure(api_key=api_key)

import folium


In [2]:
new_york_coordinates = (40.7484, -73.9857)
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(center=new_york_coordinates, zoom_level=12, layout=figure_layout)
fig.add_layer(gmaps.marker_layer([new_york_coordinates]))
fig


Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [3]:
GMAPS_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# https://developers.google.com/places/web-service/search#TextSearchRequests
MIN_USER_RATINGS = 40
MIN_RATING = 4
NRESULTS = 50
RADIUS = 3000

# home
mylocation = '40.6782308,-73.9974274'
# grand army plaza
# mylocation = '40.671872,-73.972544'
# union st and 4th ave
# mylocation ='40.677485,-73.983310'
# bay ridge
# mylocation = "40.624468,-74.0487134"
# williamsburg 
# mylocation = "40.7144609,-73.9553373"
# rankby='prominence'
rankby='distance'
# keyword='coffee'
keyword='pizza'
ltype='establishment'
#https://developers.google.com/places/web-service/supported_types

In [4]:
def get_first_page(api_key, location, **kwargs):
    """get first page of results from gmaps using api_key, location, kwargs for search spec"""
    request_url = GMAPS_URL + '?key=' + api_key
    request_url += '&location=' + location
    for name, val in kwargs.items():
        request_url += '&' + name + '=' + val
    r = requests.get(request_url)
    j = r.json()
    return j


def get_next_page(api_key, next_page_token):
    """get next search engine results page page using search token, waiting until available"""
    r = requests.get(GMAPS_URL + '?pagetoken=' + next_page_token +
                        '&key=' + api_key)
    for i in range(10):
        j = r.json()
        if not j['results']: # wait for next page to be available
            time.sleep(5)
            continue
        else:
            return j


def runquery(api_key, location, **kwargs):
    """return dataframe of all results using api_key, location, search kwargs"""
    # get first page
    j = get_first_page(api_key, location, **kwargs)
    venues_df = pd.json_normalize(j['results'])

    # get pages while additional pages available
    while 'next_page_token' in j:
        next_page_token = j['next_page_token']
        time.sleep(5)
        j = get_next_page(api_key, next_page_token)
        venues_df = venues_df.append(pd.json_normalize(j['results']))
        
    return venues_df        

In [5]:
# use either rankby or radius
gmaps_df = runquery(api_key, mylocation, keyword=keyword, ltype=ltype, rankby=rankby)
# runquery(api_key, location, keyword=keyword, ltype=ltype, radius=RADIUS)
gmaps_df = gmaps_df.loc[(gmaps_df['user_ratings_total'] >= MIN_USER_RATINGS) & (gmaps_df['rating'] >= MIN_RATING)] \
        .sort_values(['rating', 'user_ratings_total'], ascending=False) \
        .reset_index(drop=True)
gmaps_df = gmaps_df[['name', 'vicinity', 'rating', 'user_ratings_total', 'geometry.location.lat', 'geometry.location.lng']]
gmaps_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng']
# drop trailing ", Brooklyn"
gmaps_df['address'] = gmaps_df['address'].apply(lambda address: " ".join(address.split(',')[:-1]))
gmaps_df


,name,address,rating,nratings,lat,lng
0,L'Arte Della Pizza Brooklyn,"172 5th Ave, Brooklyn",4.8,71,40.677606,-73.979957
1,Pizza Moto,"338 Hamilton Ave, Brooklyn",4.7,278,40.674622,-74.000800
2,Sottocasa,"298 Atlantic Ave, Brooklyn",4.6,673,40.688307,-73.988978
3,Table 87,"473 3rd Ave, Brooklyn",4.6,498,40.670658,-73.991352
4,Dellarocco's,"214 Hicks St, Brooklyn Heights",4.6,433,40.695009,-73.996108
5,Peppino's,"469 5th Ave, Brooklyn",4.6,373,40.668186,-73.986950
6,The House of Pizza & Calzone,"132 Union St, Brooklyn",4.6,316,40.683997,-74.002281
7,Lucali,"575 Henry St, Brooklyn",4.5,1290,40.681805,-74.000293
8,La Villa Pizzeria,"261 5th Ave, Brooklyn",4.5,729,40.674325,-73.981687
9,Patsy’s Pizzeria,"450 Dean St, Brooklyn",4.5,593,40.681828,-73.976196


In [47]:
markers = [(row.lat, row.lng) for row in gmaps_df.itertuples()]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Google Rating</dt><dd>{rating}</dd>
<dt>Google Reviews</dt><dd>{nratings}</dd>
</dl>
"""
marker_info = [info_box_template.format(**row) for i, row in gmaps_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=eval(mylocation), zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [48]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]
mycoords = eval(mylocation)

venues_map = folium.Map(location=[*mycoords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
    
venues_map

In [ ]:
# yelp

In [49]:
lat, lng = mycoords
response = yelp_api.search_query(categories=keyword, latitude=lat, longitude=lng, radius=RADIUS, sort_by=rankby, limit=NRESULTS)

yelp_df = pd.json_normalize(response['businesses'])
yelp_df = yelp_df.loc[(yelp_df['review_count'] >= MIN_USER_RATINGS) & (yelp_df['rating'] >= MIN_RATING)] \
    .sort_values(['rating', 'review_count'], ascending=False) \
    .reset_index(drop=True)
display_columns = ['name', 'location.address1', 'rating', 'review_count', 'coordinates.latitude', 'coordinates.longitude', 'url']
yelp_df = yelp_df[display_columns]
yelp_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng', 'url']
yelp_df


,name,address,rating,nratings,lat,lng,url
0,Lucali,575 Henry St,4.5,1560,40.681800,-74.000240,https://www.yelp.com/biz/lucali-brooklyn-3?adj...
1,Sottocasa Pizzeria,298 Atlantic Ave,4.5,628,40.688285,-73.989006,https://www.yelp.com/biz/sottocasa-pizzeria-br...
2,Peppino's Brick Oven Pizza & Restaurant,469 5th Ave,4.5,421,40.668221,-73.986936,https://www.yelp.com/biz/peppinos-brick-oven-p...
3,Piz-zetta,90 Livingston St,4.5,288,40.691283,-73.990603,https://www.yelp.com/biz/piz-zetta-brooklyn-2?...
4,Bella Gioia,209 4th Ave,4.5,206,40.677530,-73.982721,https://www.yelp.com/biz/bella-gioia-brooklyn-...
5,Pizza Moto,338 Hamilton Ave,4.5,194,40.674622,-74.000800,https://www.yelp.com/biz/pizza-moto-brooklyn-2...
6,La Villa,261 5th Ave,4.0,522,40.674332,-73.981695,https://www.yelp.com/biz/la-villa-brooklyn?adj...
7,La Cigogne,215 Union St,4.0,356,40.683501,-73.999304,https://www.yelp.com/biz/la-cigogne-brooklyn?a...
8,Table 87 - Gowanus,473 3rd Ave,4.0,328,40.670664,-73.991371,https://www.yelp.com/biz/table-87-gowanus-broo...
9,Table 87 - Brooklyn Heights,87 Atlantic Ave,4.0,270,40.691219,-73.997345,https://www.yelp.com/biz/table-87-brooklyn-hei...


In [51]:
markers = [(lat, lng) for name, address, rating, nreviews, lat, lng, url in yelp_df.values.tolist()]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Yelp Rating</dt><dd>{rating}</dd>
<dt>Yelp Reviews</dt><dd>{nratings}</dd>
</dl>
"""

marker_info = [info_box_template.format(**row) for i, row in yelp_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=mycoords, zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [52]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*mycoords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map

In [53]:
client = foursquare.Foursquare(client_id=foursquare_id, client_secret=foursquare_secret, redirect_uri='http://streeteye.com/oauth/authorize')


In [54]:
response = client.venues.search(params={'query': keyword, 'll': "%.7f,%.7f" % mycoords})
foursquare_df = pd.json_normalize(response['venues'])
foursquare_df


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id,location.neighborhood
0,49d7e8c0f964a520795d1fe3,Giardini Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1595683641,False,363 Smith St,btwn 1st Pl & 2nd St,40.679354,-73.995358,"[{'label': 'display', 'lat': 40.679354, 'lng':...",...,United States,"[363 Smith St (btwn 1st Pl & 2nd St), Brooklyn...",23034,https://www.seamless.com/menu/giardini-gourmet...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN,NaN
1,4eb5985f2c5b53141a674e2c,Mario's Pizza & Chicken,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1595683641,False,222 Hoyt St,btwn Baltic & Butler St,40.683542,-73.990013,"[{'label': 'display', 'lat': 40.68354199999999...",...,United States,"[222 Hoyt St (btwn Baltic & Butler St), Brookl...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4a9c8342f964a5206d3720e3,Pizza Town,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1595683641,False,85 5th Ave,at Warren St.,40.680045,-73.977964,"[{'label': 'display', 'lat': 40.68004532655333...",...,United States,"[85 5th Ave (at Warren St.), Brooklyn, NY 1121...",NaN,NaN,NaN,NaN,NaN,NaN,50652136,NaN
3,4ab5c11ff964a520067620e3,Joe’s Pizza of Park Slope,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1595683641,False,483 5th Ave,at 11th St.,40.667763,-73.987500,"[{'label': 'display', 'lat': 40.66776275050942...",...,United States,"[483 5th Ave (at 11th St.), Brooklyn, NY 11215...",554833,https://www.seamless.com/menu/joes-pizza-of-pa...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN,South Slope
4,4b6f389ff964a52016e52ce3,Marks Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1595683641,False,326 Van Brunt St,Pioneer,40.678861,-74.011158,"[{'label': 'display', 'lat': 40.67886051971997...",...,United States,"[326 Van Brunt St (Pioneer), Brooklyn, NY 1123...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4d83a09c5091370403506d5b,Pizza Rustica,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1595683641,False,357 3rd St,at 5th Ave.,40.673183,-73.983490,"[{'label': 'display', 'lat': 40.67318344116211...",...,United States,"[357 3rd St (at 5th Ave.), Brooklyn, NY 11215,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,58016254d67c1565bc122795,Via Roma Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1595683641,False,445 Court St,NaN,40.678067,-73.997923,"[{'label': 'display', 'lat': 40.67806651261131...",...,United States,"[445 Court St, Brooklyn, NY 11231, United States]",345019,https://www.seamless.com/menu/via-roma-445-cou...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN,NaN
7,4a3332f3f964a520139b1fe3,Smiling Pizza Restaurant,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1595683641,False,323 7th Ave,at 9th St,40.667097,-73.981399,"[{'label': 'display', 'lat': 40.667097, 'lng':...",...,United States,"[323 7th Ave (at 9th St), Brooklyn, NY 11215, ...",66847,https://www.seamless.com/menu/smiling-pizza-re...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN,NaN
8,5428807b498e538336103286,99 Cent Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1595683641,False,255 Livingston St,at Bond St,40.688831,-73.983299,"[{'label': 'display', 'lat': 40.6888313293457,...",...,United States,"[255 Livingston St (at Bond St), Brooklyn, NY ...",1754276,https://www.seamless.com/menu/99-cents-hot-piz...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN,NaN
9,5c18213c28374e002cb23f56,Jay St. Fresh 99¢ Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1595683641,F

In [59]:
# iterate through venues to get ratings, nratings
def parse_foursquare_results(response):
    """process foursquare response, query details for each row to get rating and nratings"""
    retarray = []

    for i, venue in pd.json_normalize(response['venues']).iterrows():
        venue_id = venue['id']
        # query detailed venue info from foursquare
        venue_details = client.venues(venue_id)['venue']
        try:
            venue_name = venue['name']
            venue_address = venue['location.address']
            venue_rating = venue_details['rating']
            venue_nratings = venue_details['ratingSignals']
            venue_url = venue['delivery.url']
            venue_lat = venue['location.lat']
            venue_lng = venue['location.lng']
        except Exception as e:
            # sometimes no rating ... probably not popular enough
            print(type(e), str(e))
            # print(traceback.format_exc())
            print("No rating for %s" % venue_name)
            continue

        retarray.append([venue_name, venue_address, venue_rating, venue_nratings, venue_lat, venue_lng, venue_url])
    retdf = pd.DataFrame(retarray)
    retdf.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng', 'url']
    return retdf

foursquare_df = parse_foursquare_results(response)
foursquare_df = foursquare_df.loc[(foursquare_df['nratings'] >= MIN_USER_RATINGS) & (foursquare_df['rating'] >= MIN_RATING)] \
        .sort_values(['rating', 'nratings'], ascending=False) \
        .reset_index(drop=True)
foursquare_df

<class 'KeyError'> 'rating'
No rating for Mario's Pizza & Chicken
<class 'KeyError'> 'rating'
No rating for Pizza Rustica
<class 'KeyError'> 'rating'
No rating for 99 Cent Pizza
<class 'KeyError'> 'rating'
No rating for Jay St. Fresh 99¢ Pizza
<class 'KeyError'> 'rating'
No rating for Grandmas Boys Pizza
<class 'KeyError'> 'rating'
No rating for Pizza Slab
<class 'KeyError'> 'rating'
No rating for Texas Fried Chicken & Pizza
<class 'KeyError'> 'rating'
No rating for S&S Brooklyn Pizza
<class 'KeyError'> 'rating'
No rating for 555 Steel Oven Pizza


,name,address,rating,nratings,lat,lng,url
0,Pizza Moto,338 Hamilton Ave,9.4,188,40.674802,-74.000375,NaN
1,Roberta's Pizza,261 Moore St,9.3,4087,40.705015,-73.933617,https://www.seamless.com/menu/robertas-261-moo...
2,Prince Street Pizza,27 Prince St,9.2,1860,40.723093,-73.994527,https://www.seamless.com/menu/prince-st-pizza-...
3,Juliana's Pizza,19 Old Fulton St,9.0,1365,40.702769,-73.993616,NaN
4,Luigi's Pizza,686 5th Ave,9.0,165,40.661565,-73.993235,NaN
5,Brooklyn Pizza Market,267 Smith St,7.9,54,40.682696,-73.993191,https://www.seamless.com/menu/brooklyn-pizza-m...
6,Pizza Plus,359 7th Ave,7.4,59,40.665947,-73.982361,https://www.seamless.com/menu/pizza-plus-359-7...
7,Pizza Town,85 5th Ave,7.4,51,40.680045,-73.977964,NaN
8,2 Bros. Pizza,395 Flatbush Ave,7.2,62,40.689394,-73.981125,NaN
9,Giardini Pizza,363 Smith St,7.1,84,40.679354,-73.995358,https://www.seamless.com/menu/giardini-gourmet...


In [57]:
markers = [(lat, lng) for name, address, rating, nreviews, lat, lng, url in foursquare_df.values]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Foursquare Rating</dt><dd>{rating}</dd>
<dt>Foursquare Reviews</dt><dd>{nratings}</dd>
</dl>
"""
marker_info = [info_box_template.format(**d_item) for i, d_item in foursquare_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=mycoords, zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [58]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*mycoords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map

In [ ]:
# improve search, get williamsburg pizza in there for all searches
# merge dataframes with dedupe
# https://github.com/dedupeio/dedupe
# https://sites.google.com/site/anhaidgroup/projects/magellan
# https://github.com/datamade/dedupe

In [75]:
pd.set_option('display.max_rows', None)

venues_df = pd.concat([gmaps_df, yelp_df, foursquare_df]).reset_index(drop=True)
venues_df.sort_values('name')

,name,address,rating,nratings,lat,lng,url
36,2 Bros. Pizza,395 Flatbush Ave Ext #5321,4.1,451,40.689439,-73.980940,NaN
77,2 Bros. Pizza,395 Flatbush Ave,7.2,62,40.689394,-73.981125,NaN
42,99 Cents Hot Pizza,255 Livingston St,4.0,119,40.688824,-73.983285,NaN
24,Artichoke Basille's Pizza,59 5th Ave,4.3,930,40.680943,-73.977142,NaN
49,Bella Gioia,209 4th Ave,4.5,206,40.677530,-73.982721,https://www.yelp.com/biz/bella-gioia-brooklyn-...
32,Big Daddy's Pizza,68 Lorraine St,4.3,63,40.674366,-74.007135,NaN
63,Brado,155 Atlantic Ave,4.0,134,40.690557,-73.995479,https://www.yelp.com/biz/brado-brooklyn-3?adju...
14,Brado,155 Atlantic Ave,4.5,244,40.690645,-73.994911,NaN
38,Brooklyn Pizza Market,267 A Smith St,4.1,122,40.682663,-73.993080,NaN
74,Brooklyn Pizza Market,267 Smith St,7.9,54,40.682696,-73.993191,https://www.seamless.com/menu/brooklyn-pizza-m...


In [76]:
!pip install dedupe


     |████████████████████████████████| 60 kB 2.2 MB/s eta 0:00:011
     |████████████████████████████████| 172 kB 6.5 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 9.6 MB/s  eta 0:00:01
     |████████████████████████████████| 874 kB 7.2 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 10.9 MB/s eta 0:00:01
     |████████████████████████████████| 144 kB 10.2 MB/s eta 0:00:01
     |████████████████████████████████| 192 kB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 204 kB 11.2 MB/s eta 0:00:01
